<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hyperparameters search 
!pip install ray

In [ ]:
import ray
from ray import tune

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

In [ ]:
train_dataset = dataset["train"].shard(index=1, num_shards=10)

In [ ]:
   config = {
       "learning_rate": tune.choice([3e-5,3e-4,3e-3]),
       "per_device_train_batch_size": tune.choice([6,8,12,16,24]),
       "per_device_test_batch_size": tune.choice([6,8,12,16,24]),
       "num_train_epochs": tune.choice([6,12,18,24]),
       "weight_decay": tune.choice([0.1,0.15,0.2,0.25,0.3]),
       "gradient_accumulation_steps": tune.choice([1,2,3,4,5,6]),
          "wandb":{
            "project": "IR_LDC",
            "api_key": 'XXX'
        }
}

In [ ]:
!git clone https://github.com/danielsaggau/IR_LDC.git
%cd IR_LDC

Cloning into 'IR_LDC'...
remote: Enumerating objects: 1730, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 1730 (delta 21), reused 0 (delta 0), pack-reused 1693
Receiving objects: 100% (1730/1730), 3.91 MiB | 12.93 MiB/s, done.
Resolving deltas: 100% (1094/1094), done.
/content/IR_LDC


In [ ]:
!pip install -r requirements.txt

In [ ]:
!python /content/IR_LDC/model/SCOTUS/scotus_clean.py \
    --output_dir logs/output_1 \
    --model_type 'max' \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --gradient_accumulation_steps 1 \
    --metric_for_best_model "f1-micro" \
    --greater_is_better 1 \
    --report_to 'wandb'

In [ ]:
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    eval_dataset=tokenized_data['test'],
    train_dataset=tokenized_data["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
      )

NameError: ignored

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

from ray import tune
tune.run(
    trainer,
    config=config)

NameError: ignored

In [ ]:
scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="eval_acc",
        mode="max",
        perturbation_interval=1,
        hyperparam_mutations={
            "weight_decay": tune.uniform(0.0, 0.3),
            "learning_rate": tune.uniform(1e-3, 5e-5),
            "per_device_train_batch_size": [6, 12,18, 24],
        },
        "wandb":{
            "project": "IR_LDC",
            "api_key": 'fd6f7deb3126d40be9abf77ee753bf45f00e2a9a'
        }
    )

reporter = CLIReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "per_device_train_batch_size": "train_bs/gpu",
            "num_train_epochs": "num_epochs",
        },
        metric_columns=[ ,"eval_loss", "epoch", "training_iteration"],
    )

trainer.hyperparameter_search(
        hp_space=lambda _: tune_config,
        backend="ray",
        n_trials=2
        ,
        resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
        scheduler=scheduler,
        keep_checkpoints_num=1,
        checkpoint_score_attr="training_iteration",
        stop={"training_iteration": 1} if smoke_test else None,
        progress_reporter=reporter,
        local_dir="~/ray_results/",
        name="tune_transformer_pbt",
        log_to_file=True,
    )